In [1]:
import logging
from collections import ChainMap

logging.basicConfig(level=logging.INFO)

DEFAULT_VALUES = {
    "queue": "default_queue",
    "countdown": "3600", # 60 * 60 minutes
}


def validate_request_inputs(values: ChainMap) -> None:
    # We make sure task_id is computed and not set by the user
    if "user_id" in values.maps[-1]:
        raise ValueError("'user_id' cannot be set by user.")
    # task_id can be overwritten by user if needed.
    assert values["task_id"].endswith("id")

def process_request(**kwargs):
    values = ChainMap({}, DEFAULT_VALUES, kwargs);
    values["user_id"] = "computed_user_id"
    values["task_id"] = values.get("task_id", "computed_task_id")
    
    validate_request_inputs(values)
    
    logging.info("processing with values: %s", dict(values))
    #return create_request(values)
    
    

When we set a value in a `ChainMap` the value is set to the first dictionary given to the constructor (line 20). In this example every value will be set to an empty dictionary.

But, key lookup is done in reverse other. Here, a key will be first looked up on `kwargs` then on `DEFAULT_VALUES` then on `{}`.

In [2]:
process_request(user="username", message="some message")

INFO:root:processing with values: {'user': 'username', 'message': 'some message', 'queue': 'default_queue', 'countdown': '3600', 'user_id': 'computed_user_id', 'task_id': 'computed_task_id'}


User cannot specify a `user_id`

In [3]:
process_request(user_id="custom")

ValueError: 'user_id' cannot be set by user.

But, user can specify a `task_id`

In [4]:
process_request(task_id="my_task_id")

INFO:root:processing with values: {'task_id': 'my_task_id', 'queue': 'default_queue', 'countdown': '3600', 'user_id': 'computed_user_id'}
